In [2]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
import math as m

# Cargar el modelo YOLOv8-Pose
model_path = "E:/Archivos Universidad/CICLOS UPC/CICLO 6/Procesamiento de imagenes/TF/runs/pose/train10/weights/best.pt"
model = YOLO(model_path)

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Función para calcular ángulo entre tres puntos
def calculate_angle(a, b, c):
    a = np.array(a)  # Primer punto
    b = np.array(b)  # Segundo punto (vértice)
    c = np.array(c)  # Tercer punto
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Función para calcular el ángulo entre dos puntos
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 - y1) * (-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180 / m.pi) * theta
    return degree

# Inicializar cámara
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se puede abrir la cámara.")
    exit()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    image_pose = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_pose = cv2.flip(image_pose, 1)
    image_pose.flags.writeable = False
    results_pose = pose.process(image_pose)
    image_pose.flags.writeable = True
    image_pose = cv2.cvtColor(image_pose, cv2.COLOR_RGB2BGR)

    # Detección de postura con MediaPipe
    if results_pose.pose_landmarks:
        landmarks = results_pose.pose_landmarks.landmark

        # Dibujar todos los puntos clave
        for landmark in landmarks:
            x = int(landmark.x * image_pose.shape[1])
            y = int(landmark.y * image_pose.shape[0])
            cv2.circle(image_pose, (x, y), 5, (0, 255, 0), -1)  # Dibujar círculo en el punto clave

        # Puntos de interés para la postura
        earL = 8  # Oído izquierdo seria 7 pero mi cam reocnoce 8 
        shouldL = 12  # Hombro izquierdo seria 11 pero micam reconoce 12
        hipsL = 24  # Cadera izquierda seria 23 pero mi cam reconoce 24

        if landmarks[earL] and landmarks[shouldL] and landmarks[hipsL]:
            h, w, c = image_pose.shape
            earX = int(landmarks[earL].x * w)
            earY = int(landmarks[earL].y * h)
            SLpX = int(landmarks[shouldL].x * w)
            SLpY = int(landmarks[shouldL].y * h)
            hLX = int(landmarks[hipsL].x * w)
            hLY = int(landmarks[hipsL].y * h)

            # Dibujar puntos y líneas para la postura
            cv2.circle(image_pose, (SLpX, earY), 10, (255, 255, 0), -1)
            cv2.circle(image_pose, (earX, earY), 10, (255, 255, 0), -1)
            cv2.circle(image_pose, (SLpX, SLpY), 10, (255, 255, 0), -1)
            cv2.circle(image_pose, (hLX, hLY), 10, (255, 255, 0), -1)

            cv2.line(image_pose, (earX, earY), (SLpX, SLpY), (0, 0, 255), 9)
            cv2.line(image_pose, (SLpX, SLpY), (hLX, hLY), (0, 0, 255), 9)
            cv2.line(image_pose, (SLpX, SLpY), (SLpX, earY), (0, 0, 255), 9)

            angulo = findAngle(SLpX, SLpY, earX, earY)
            cv2.putText(image_pose, f"Inclinacion del cuello: {int(angulo)} grados", (20, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Detectar mala postura 
            if angulo > 30:
                cv2.putText(image_pose, "MALA POSTURA", (20, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image_pose, "BUENA POSTURA", (20, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Detección de objetos con YOLOv8
    results_yolo = model(frame)[0]

    for result in results_yolo.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        # Color basado en la clase
        color = (0, 255, 0) if class_id == 1 else (0, 0, 255)
        cv2.rectangle(image_pose, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
        cv2.putText(image_pose, f'{model.names[int(class_id)].upper()}: {score:.2f}', (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    cv2.imshow('Detección de objeto y postura', image_pose)

    # Detener la ejecución si se presiona 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 39.9ms
Speed: 6.1ms preprocess, 39.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 31.3ms
Speed: 0.8ms preprocess, 31.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 28.7ms
Speed: 0.0ms preprocess, 28.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 36.1ms
Speed: 0.0ms preprocess, 36.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 38.1ms
Speed: 0.0ms preprocess, 38.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 36.5ms
Speed: 0.0ms preprocess, 36.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 22.2ms
Speed: 0.0ms preprocess, 22.2ms inference, 12.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 33.2ms
Speed: 0.0ms preprocess, 33.2ms 